# Story-writing with BLOOM

SPDX-License-Identifier: 0BSD

In [1]:
from complete import Completer

In [2]:
bloom1 = Completer("""
Sometimes I marvel at how the sky contains no advertisements. Oh, sometimes
someone goes up in a little plane and skywrites a birthday greeting. But the
sky... you don't have to watch an ad to see it.

When people learn I fought in the war to free the sky, they say, "Thank you for
""", temperature=1.0, top_k=30)

In [3]:
bloom1()

Sometimes I marvel at how the sky contains no advertisements. Oh,
sometimes someone goes up in a little plane and skywrites a birthday
greeting. But the sky... you don't have to watch an ad to see it.

When people learn I fought in the war to free the sky, they say,
"Thank you for defending God's playground." I say, "No, no... it's our
playground. If the sky had a fence and a gate, I wouldn't have been a
part of this mission. So we don't have fences. And we don't have
gates. We let people wander."

Sometimes, when I look around the world, I think: is there any place
on Earth where children can play without their toys and prizes and
cars and planes and money? Is there any place on Earth where children
live without a television, where they don't watch an ad all day long?
And then I look up at the sky and say... there is no such thing as a
"commercial sky." If the sky is God's playground, and the sky is also
our playground, then the sky belongs to children. And children don't
live to adve

In [4]:
bloom2 = Completer("""
The color enforcers came around again. They ripped the cheese off our freshly
baked pizza. The customers dropped their food and ran out.

"Aha!" said the biggest, scariest henchman. "This sauce is red! We license the
color red to you, but you haven't been paying the license fees." I quaked in
fear. My grandfather had gotten both his knees busted by color enforcers.
""")

In [5]:
bloom2()

The color enforcers came around again. They ripped the cheese off our
freshly baked pizza. The customers dropped their food and ran out.

"Aha!" said the biggest, scariest henchman. "This sauce is red! We
license the color red to you, but you haven't been paying the license
fees." I quaked in fear. My grandfather had gotten both his knees
busted by color enforcers. He'd never been able to work again. And my
grandma was already cold in her grave.

I thought about my son's pizza. It was brown, which wasn't on the
approved list! I was about to cry when one of the enforcers looked at
the box.

My grandpa would have been proud of me. I filed a complaint with the
government. The new local representative, a legislator named Philando,
listened to my story and saw the injustice. He immediately brought a
bill to Congress, demanding that the color enforcers turn over their
licenses to the manufacturers. Now if the manufacturers had to pay a
fee for each color, it would be fair. The color enforcer

In [6]:
bloom3 = Completer("""
Beverly understood how all the women in her family had fallen in love with this
energy ghost who lived in a lamp and fed parasitically off their vital energy
while making every carnal dream come to life. As a spacefaring physician,
""")

In [7]:
bloom3()

Beverly understood how all the women in her family had fallen in love
with this energy ghost who lived in a lamp and fed parasitically off
their vital energy while making every carnal dream come to life. As a
spacefaring physician, she had seen how the energy source was used to
fuel the ships of the mindless hordes of the Draconian Empire, but she
had never seen it used for something as benign as a lamp. She had
studied the true nature of the entity, discovering that it was created
from the Quarantine Zone as part of a biological defense shield used
to quarantine the Zone from the rest of the universe. It was a part of
the living energy of the Zone itself, and had been created from the
same material as the Zone. It was programmed to feed on energy and was
designed to be destroyed if it left the Zone. It could never be
killed, only disarmed. Its power was so great that it was all but
impossible to do so, which was why the Draconian Empire had been so
eager for it to get out of the Zone.

In [8]:
# From https://aclanthology.org/2020.acl-main.463.pdf, Appendix A.
bloom4 = Completer("""
Robinson cried out, “Help! I’m being chased by a bear! All I have is these
sticks. What should I do?” The survivalist said, “Here’s what you can do.
""")

In [9]:
bloom4()

Robinson cried out, “Help! I’m being chased by a bear! All I have is
these sticks. What should I do?” The survivalist said, “Here’s what
you can do. Stand still and in the meantime, take these sticks.” The
man did just that and the bear eventually went away.
